<a href="https://colab.research.google.com/github/shreyasat27/quantupid_10/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 12.2 MB/s eta 0:00:00


In [ ]:
import numpy as np

def hamiltonian_matrix_generate(spring_constant, mass_diagonal_matrix, position_vector, initial_velocity_vector):
    # Calculate mew
    mew = np.sqrt(spring_constant) * position_vector
    print(mew)

    # Construct B_dagger
    B_dagger = np.zeros((len(mass_diagonal_matrix), len(mew)))
    B_dagger[:len(position_vector), :] = np.sqrt(mass_diagonal_matrix) * position_vector
    for i in range(len(position_vector)):
        for j in range(i+1, len(position_vector)):
            B_dagger[j+len(position_vector), i] = np.sqrt(2 * spring_constant[i, j]) * position_vector[j]

    # Find B
    B = np.conjugate(B_dagger).T

    # Construct Hamiltonian matrix
    H = np.block([[np.zeros((len(mass_diagonal_matrix), len(mass_diagonal_matrix))), B],
                  [B_dagger, np.zeros((len(mass_diagonal_matrix), len(mass_diagonal_matrix)))]])

    return H

# Given parameters
spring_constant = np.array([[0, 4], [4, 0]])
position_vector = np.array([[2], [4]])  # Column vector
mass_diagonal_matrix = np.array([[4, 0], [0, 9]])
initial_velocity_vector = np.array([[2], [5]])  # Column vector

# Generate Hamiltonian matrix
H = hamiltonian_matrix_generate(spring_constant, mass_diagonal_matrix, position_vector, initial_velocity_vector)
print("Hamiltonian matrix:")
print(H)


[[0. 4.]
 [8. 0.]]


IndexError: index 3 is out of bounds for axis 0 with size 2

In [ ]:
spring_constant=np.array([0,4],[4,0])
position_vector=np.array([2,4])
mass_diagonal_matrix=np.array([4,0],[0,9])
initial_velocity_matrix=np.array([2,5])



In [ ]:
import numpy as np
import pennylane as qml

# Define the matrix A
A = np.array([[-2, -2+1j, -2, -2],
              [-2-1j, 0, 0, -1],
              [-2, 0, -2, -1],
              [-2, -1, -1, 0]])

# Decompose A into Pauli terms
H = qml.pauli_decompose(A)

# Define the file path
file_path = "pauli_decomposition.txt"

# Convert Hamiltonian to a list of terms
# terms = list(H)

# # Save the decomposition to the text file
# with open(file_path, "w") as file:
#     for term in terms:
#         file.write(f"{term[1]} {term[0]}\n")

# print(f"Pauli decomposition saved to {file_path}")


TypeError: 'Hamiltonian' object is not iterable

In [ ]:
import numpy as np
import qiskit
import scipy
from scipy import linalg
from qiskit.quantum_info import SparsePauliOp, Operator, Pauli
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.synthesis import LieTrotter, SuzukiTrotter
from qiskit import transpile

######## Reading the data
# './hamiltonian_string_raw.txt' should contain the Hamiltonian
# posted to the competition forum
hamiltonian_terms = []
with open('/content/hamiltonian_string_raw.txt', 'r') as f:
    for line in f:
        line_remove_newlines = line.replace('\n', '')
        line_remove_spaces = ''.join(line_remove_newlines.split(' '))
        if line_remove_spaces == '':
            continue
        line_items = line_remove_spaces.split('*')
        assert len(line_items) == 2
        coef, pauli_term = line_items
        hamiltonian_terms.append((float(coef), pauli_term))

######## Sort by absolute value of the coefficient.
# Terms with high absolute value of the coefficient are more important

hamiltonian_terms.sort(key=lambda term: abs(term[0]))
for term in hamiltonian_terms:
    print(term[0], '\t', term[1])
print(f'Hamiltonian has {len(hamiltonian_terms)} terms')

######## Compute U_perfect := the ideal unitary that we should simulate
H = SparsePauliOp.from_list([(item[1], item[0]) for item in hamiltonian_terms])
U_perfect = scipy.linalg.expm(-1j * H.to_matrix())

######## Divide the hamiltonian into two parts:
# Z_part, which contains terms with Paulis made up only from I and Z operators
# R_part, which contains all the rest
Z_part = SparsePauliOp.from_list([(item[1], item[0]) for item in hamiltonian_terms if 'X' not in item[1] and 'Y' not in item[1]])
R_part = SparsePauliOp.from_list([(item[1], item[0]) for item in hamiltonian_terms if 'X' in item[1] or 'Y' in item[1]])

######## Helper function which takes a SparsePauliOp, time t,
# and returns a circuit that trotterizes the e^-it*sparsepauliop
def firstorder(sparsepauliop, t):
    UT = PauliEvolutionGate(sparsepauliop, t)
    trotter = LieTrotter(reps=1, cx_structure='fountain')
    return trotter.synthesize(UT)

######## A greedy sorter for a SparsePauliOp.
# Say, we have some Pauli terms IXXII, IIIZX. Then they can be trotterized efficiently,
# i.e. each works on different set of qubits so they can be executed in parallel.
# We try to order the terms so that such occurences are maximized.
def greedy_sorter(sparsepauliop):
    s = list(item for item in sparsepauliop)
    new_order = []
    while len(s) > 0:
        coll = []
        coll_used = set()
        added = True
        while added:
            added = False
            for item in s:
                item_pauli = item.paulis[0]
                item_used = set(i for i in range(len(item_pauli)) if item_pauli[i] != Pauli('I'))
                if len(coll_used.intersection(item_used)) == 0:
                    coll.append(item)
                    coll_used = coll_used.union(item_used)
                    s.remove(item)
                    added = True
        new_order = new_order + coll
    ret = new_order[0]
    for j in range(1, len(new_order)):
        ret = ret + new_order[j]
    return ret

######## Let us define two smaller circuits:
# Z_part_evol_half = a circuit that implements the unitary e^-0.5*Z_part
# R_part_evol_one = a circuit that trotterizes all the R_parts
# The final circuit will consist of three parts:
# (Z_part_evol_half) (R_part_evol_one) (Z_part_evol_half)
# It turns out that such circuit achieves an accuracy well below the required 0.1 threshold.
# So we simply ignore the terms from R_part with lowest abs. value of the coefficient,
# losing some of the precision but making the circuit shorter.
Z_part_evol_half = firstorder(greedy_sorter(Z_part), 0.5)
R_part_evol_one = firstorder(greedy_sorter(R_part[120:]), 1)
circ = Z_part_evol_half.compose(R_part_evol_one).compose(Z_part_evol_half)
circ_transpiled = transpile(circ, optimization_level=2, basis_gates=['x', 'y', 'z', 'rz', 'ry', 'rx', 'h', 'u', 'cx'])
# circ_transpiled.qasm3(filename='./solution.qasm')

######## Finally, we will check the precision of the produced circuit.
circ_loaded = circ_transpiled
U = np.asmatrix(qiskit.quantum_info.Operator(circ_loaded))

# Note that special care must be taken, since transpiling to QASM in qiskit
# messes up the global phase.
assert np.allclose(abs(U[0, 0]), 1)
U_fixed_phase = U / U[0, 0]
print(f"Operator norm error: {np.linalg.norm(U_perfect - U_fixed_phase, ord=2)}")
print(f"Circuit depth: {circ_loaded.depth()}")

# Results of the run:
# Operator norm error: 0.0986
# Circuit depth: 697


-0.0003518893528389501 	 IZIIIYZZYI
-0.0003518893528389501 	 IZIIIXZZXI
-0.0003518893528389501 	 IIZIIYZZYI
-0.0003518893528389501 	 IIZIIXZZXI
-0.0003518893528389501 	 YZZYIIZIII
-0.0003518893528389501 	 XZZXIIZIII
-0.0003518893528389501 	 YZZYIIIZII
-0.0003518893528389501 	 XZZXIIIZII
-0.0011822832324725804 	 IZIIIIIIYY
-0.0011822832324725804 	 IZIIIIIIXX
-0.0011822832324725804 	 IIZIIIIIYY
-0.0011822832324725804 	 IIZIIIIIXX
-0.0011822832324725804 	 IIIYYIZIII
-0.0011822832324725804 	 IIIXXIZIII
-0.0011822832324725804 	 IIIYYIIZII
-0.0011822832324725804 	 IIIXXIIZII
-0.00154067008970742 	 IIIZIIIIYY
-0.00154067008970742 	 IIIZIIIIXX
-0.00154067008970742 	 IIIYYIIIZI
-0.00154067008970742 	 IIIXXIIIZI
0.00211113766859809 	 YZZYIIIIYY
0.00211113766859809 	 XZZXIIIIYY
0.00211113766859809 	 YZZYIIIIXX
0.00211113766859809 	 XZZXIIIIXX
0.00211113766859809 	 IIIYYYZZYI
0.00211113766859809 	 IIIXXYZZYI
0.00211113766859809 	 IIIYYXZZXI
0.00211113766859809 	 IIIXXXZZXI
-0.0021498576488650843 	

In [ ]:
from qiskit.visualization import circuit_drawer

# Draw the circuit
circuit_drawer(circ, output='text')


┌─────────────────────┐┌─────────────────────┐┌─────────────────────┐»
q_0: ┤0                    ├┤0                    ├┤0                    ├»
     │                     ││                     ││                     │»
q_1: ┤1                    ├┤1                    ├┤1                    ├»
     │                     ││                     ││                     │»
q_2: ┤2                    ├┤2                    ├┤2                    ├»
     │                     ││                     ││                     │»
q_3: ┤3                    ├┤3                    ├┤3                    ├»
     │                     ││                     ││                     │»
q_4: ┤4                    ├┤4                    ├┤4                    ├»
     │  exp(it IIIZZIIIII) ││  exp(it IIIIIZIIZI) ││  exp(it ZZIIIIIIII) │»
q_5: ┤5                    ├┤5                    ├┤5                    ├»
     │                     ││                     ││                     │»
q_6: ┤6                    ├┤6                    ├┤6                    ├»
     │                     ││                     ││                     │»
q_7: ┤7                    ├┤7                    ├┤7                    ├»
     │                     ││                     ││                     │»
q_8: ┤8                    ├┤8                    ├┤8                    ├»
     │                     ││                     ││                     │»
q_9: ┤9                    ├┤9                    ├┤9                    ├»
     └─────────────────────┘└─────────────────────┘└─────────────────────┘»
«     ┌─────────────────────┐┌─────────────────────┐┌─────────────────────┐»
«q_0: ┤0                    ├┤0                    ├┤0                    ├»
«     │                     ││                     ││                     │»
«q_1: ┤1                    ├┤1                    ├┤1                    ├»
«     │                     ││                     ││                     │»
«q_2: ┤2                    ├┤2                    ├┤2                    ├»
«     │                     ││                     ││                     │»
«q_3: ┤3                    ├┤3                    ├┤3                    ├»
«     │                     ││                     ││                     │»
«q_4: ┤4                    ├┤4                    ├┤4                    ├»
«     │  exp(it IIIIIIZIIZ) ││  exp(it IIZIIIIZII) ││  exp(it IIIIIIIIII) │»
«q_5: ┤5                    ├┤5                    ├┤5                    ├»
«     │                     ││                     ││                     │»
«q_6: ┤6                    ├┤6                    ├┤6                    ├»
«     │                     ││                     ││                     │»
«q_7: ┤7                    ├┤7                    ├┤7                    ├»
«     │                     ││                     ││                     │»
«q_8: ┤8                    ├┤8                    ├┤8                    ├»
«     │                     ││                     ││                     │»
«q_9: ┤9                    ├┤9                    ├┤9                    ├»
«     └─────────────────────┘└─────────────────────┘└─────────────────────┘»
«     ┌─────────────────────┐┌─────────────────────┐┌─────────────────────┐»
«q_0: ┤0                    ├┤0                    ├┤0                    ├»
«     │                     ││                     ││                     │»
«q_1: ┤1                    ├┤1                    ├┤1                    ├»
«     │                     ││                     ││                     │»
«q_2: ┤2                    ├┤2                    ├┤2                    ├»
«     │                     ││                     ││                     │»
«q_3: ┤3                    ├┤3                    ├┤3                    ├»
«     │                     ││                     ││                     │»
«q_4: ┤4                    ├┤4                    ├┤4                    ├»
«     │  exp(it IIIIII